# Vehicle Classification

## Evaluate your model


In [2]:
import sys
sys.path.append("..")

import os

from sklearn.metrics import accuracy_score, classification_report

from utils import utils
from models import resnet_50


2022-11-21 16:00:11.138965: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 16:00:11.389114: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-21 16:00:11.389134: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-21 16:00:11.416567: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-21 16:00:12.087812: W tensorflow/stream_executor/platform/de

## Experiment settings

Set here the two variables in the following way:

- **CONFIG_YML:** assign the path to the config.yml file used for the experiment you want to evaluate
- **WEIGHTS:** assign the path to the model weights (.h5 file) you want to evaluate


In [2]:
# Modify this
CONFIG_YML = "../experiments/exp_12/config01.yml"
WEIGHTS = "../experiments/exp_12/model.51-2.9506.h5"

In [3]:
# (!) Don't touch this cell, will load the config file specified before
config = utils.load_config(CONFIG_YML)
config

{'seed': 123,
 'data': {'directory': '../data/car_ims_v1/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 32},
 'model': {'weights': 'imagenet',
  'input_shape': [224, 224, 3],
  'classes': 196,
  'dropout_rate': 0.1,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal_and_vertical',
    'name': 'random_flip_01'},
   'random_rotation': {'factor': 0.2, 'name': 'random_rotation_01'},
   'random_zoom': {'height_factor': 0.2,
    'width_factor': 0.2,
    'name': 'random_zoom_01'}}},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.00025}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 120,
  'callbacks': {'model_checkpoint': {'filepath': '/home/app/src/experiments/exp_12/model.{epoch:02d}-{val_loss:.4f}.h5',
    'save_best_only': True},
   'tensor_board': {'log_dir': '/home/app/src/experiments/exp_12/logs'},
   'early_stopping': {'monitor': 'val_loss',
    

In [4]:
# (!) Don't touch this cell, will use the config file to infer the class names
#     and also to load the corresponding testing dataset.
#     If you get an error, you must review your data/code.

MODEL_CLASSES = utils.get_class_names(config)

if len(MODEL_CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')

if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(MODEL_CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

## Load your model

Use `resnet_50.create_model()` and remember to properly setup the model weights!

Assign the model to the variable `cnn_model`.


In [5]:
# Complete this
# TODO
     # **config['model']
cnn_model=resnet_50.create_model(WEIGHTS)
# It should print your model correctly
#print(cnn_model)
print(cnn_model.summary())

2022-11-20 11:42:14.605652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-20 11:42:14.605674: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-20 11:42:14.605687: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (manuelquiros): /proc/driver/nvidia/version does not exist
2022-11-20 11:42:14.606561: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

## Get predictions from testing dataset

In [7]:
# (!) Don't touch this cell, will use the loaded model and
#     the function utils.predict_from_folder() to get 
#     model predictions and the corresponding true labels
#     so we can measure the accuracy

predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size= (224, 224),#config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)
print(predictions)
print(labels)
if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


../data/car_ims_v1/test/Bugatti Veyron 16.4 Convertible 2009/003624.jpg
1/1 [==============================] - 0s 45ms/step
1
1
../data/car_ims_v1/test/Bugatti Veyron 16.4 Convertible 2009/003618.jpg
1/1 [==============================] - 0s 56ms/step
2
2
../data/car_ims_v1/test/Bugatti Veyron 16.4 Convertible 2009/003636.jpg
1/1 [==============================] - 0s 51ms/step
3
3
../data/car_ims_v1/test/Bugatti Veyron 16.4 Convertible 2009/003627.jpg
1/1 [==============================] - 0s 50ms/step
4
4
../data/car_ims_v1/test/Bugatti Veyron 16.4 Convertible 2009/003615.jpg
1/1 [==============================] - 0s 45ms/step
5
5
../data/car_ims_v1/test/Bugatti Veyron 16.4 Convertible 2009/003623.jpg
1/1 [==============================] - 0s 44ms/step
6
6
../data/car_ims_v1/test/Bugatti Veyron 16.4 Convertible 2009/003621.jpg
1/1 [==============================] - 0s 51ms/step
7
7
../data/car_ims_v1/test/Bugatti Veyron 16.4 Convertible 2009/003646.jpg
1/1 [===========================

In [8]:
predictions

array(['0.9949805', '0.2528768', '0.2779643', ..., '0.5436689',
       '0.13546868', '0.11800351'], dtype='<U32')

In [9]:
labels

array(['Bugatti Veyron 16.4 Convertible 2009', 'BMW M6 Convertible 2010',
       'Mercedes-Benz E-Class Sedan 2012', ..., 'Volvo XC90 SUV 2007',
       'Isuzu Ascender SUV 2008', 'Volvo XC90 SUV 2007'], dtype='<U54')

In [10]:
# (!) Don't touch this cell

print(classification_report(y_true=labels, y_pred=predictions))


                                                        precision    recall  f1-score   support

                                           0.043768466       0.00      0.00      0.00       0.0
                                           0.048299983       0.00      0.00      0.00       0.0
                                            0.05000493       0.00      0.00      0.00       0.0
                                            0.05077828       0.00      0.00      0.00       0.0
                                            0.05211955       0.00      0.00      0.00       0.0
                                            0.05256354       0.00      0.00      0.00       0.0
                                            0.05260353       0.00      0.00      0.00       0.0
                                           0.053086683       0.00      0.00      0.00       0.0
                                           0.053294618       0.00      0.00      0.00       0.0
                                       

/home/manuelquiros/Documents/VENVS/sprint05/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/manuelquiros/Documents/VENVS/sprint05/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/manuelquiros/Documents/VENVS/sprint05/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

In [11]:
# (!) Don't touch this cell

acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.0000!


ValueError: Your model accuracy is too low :(
You can do it better! :)